In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PC
import pinecone as pc
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore


In [2]:
PINECONE_API_KEY = "54a50604-ce10-49b5-96c7-0438b971d14d"
PINECONE_API_ENV = "gcp-starter"

In [3]:
# extract data from PDF
def load_pdf(data):
   loader = DirectoryLoader(data,
                    glob = "*.pdf",
                    loader_cls=PyPDFLoader)
   documents = loader.load()
   
   return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
extracted_data

[Document(page_content='Octatrack MKII\nUser Manual', metadata={'source': 'data/Octatrack-MKII-User-Manual_ENG_OS1.40A_210414.pdf', 'page': 0}),
 Document(page_content='FCC compliance statement\nThis device complies with part 15 of the FCC rules. Operation is subject to the following two conditions: \n(1) This device may not cause harmful interference, and (2) this device must accept any interference \nreceived, including interference that may cause undesired operation.\nNOTE: This equipment has been tested and found to comply with the limits for a Class B digital device, \npursuant to Part 15 of the FCC Rules. These limits are designed to provide reasonable protection \nagainst harmful interference in a residential installation. This equipment generates, uses and can \nradiate radio frequency energy and, if not installed and used in accordance with the instructions, may \ncause harmful interference to radio communications. However, there is no guarantee that interference \nwill not oc

In [6]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunks", len(text_chunks))

length of my chunks 938


In [8]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("hello world")
print ("Length", len(query_result))

Length 384


In [12]:
query_result

[-0.03447726368904114,
 0.03102320060133934,
 0.00673496350646019,
 0.0261089988052845,
 -0.03936200216412544,
 -0.1603025197982788,
 0.06692398339509964,
 -0.006441466510295868,
 -0.04745054244995117,
 0.01475883275270462,
 0.07087534666061401,
 0.05552753433585167,
 0.019193334504961967,
 -0.026251306757330894,
 -0.010109467431902885,
 -0.026940535753965378,
 0.02230745740234852,
 -0.022226646542549133,
 -0.1496926248073578,
 -0.017493095248937607,
 0.007676240056753159,
 0.054352279752492905,
 0.0032544971909374,
 0.031725987792015076,
 -0.08462139964103699,
 -0.0294059868901968,
 0.05159568414092064,
 0.04812408983707428,
 -0.0033148066140711308,
 -0.058279234915971756,
 0.04196927696466446,
 0.02221076935529709,
 0.128188818693161,
 -0.02233896777033806,
 -0.011656266637146473,
 0.06292840838432312,
 -0.032876282930374146,
 -0.09122607111930847,
 -0.03117534890770912,
 0.052699554711580276,
 0.04703483358025551,
 -0.08420303463935852,
 -0.030056165531277657,
 -0.020744826644659042

In [13]:
import os
from pinecone import Pinecone

In [14]:
from pinecone import ServerlessSpec
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [15]:
index_name = 'octatrack'

In [19]:
from pinecone import Pinecone

pc = Pinecone(api_key="54a50604-ce10-49b5-96c7-0438b971d14d")
index = pc.Index("octatrack")

In [20]:
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'octa2-h3u6h7t.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'octa2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 384,
              'host': 'octatrack-h3u6h7t.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'octatrack',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [21]:
import time

index = pc.Index(index_name)
time.sleep(1)

In [22]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 938}},
 'total_vector_count': 938}

In [23]:
from langchain_pinecone import PineconeVectorStore


In [24]:
index_name = "octatrack"

In [25]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [26]:
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [41]:
print(text_chunks[0].page_content)

Octatrack MKII
User Manual


In [27]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "what are parts?"

docs = docsearch.similarity_search(query, k = 2)
print("Result", docs)

Result [Document(page_content='want to keep them, it is very important to save the project. Use the SAVE TO NEW com -\nmand for this. Read more about this command in section “8.4.1 PROJECT” on page 31.\n10.2 PARTS\nParts are very useful for adding variations to the patterns of a bank. A part consists of machine, sample and \neffect assignments along with track parameter settings and up to 16 scenes. Each pattern is linked to a part \nand one part is thus always active. Each bank hosts 4 parts.', metadata={'page': 51.0, 'source': 'data/Octatrack-MKII-User-Manual_ENG_OS1.40A_210414.pdf'}), Document(page_content='want to keep them, it is very important to save the project. Use the SAVE TO NEW com -\nmand for this. Read more about this command in section “8.4.1 PROJECT” on page 31.\n10.2 PARTS\nParts are very useful for adding variations to the patterns of a bank. A part consists of machine, sample and \neffect assignments along with track parameter settings and up to 16 scenes. Each patte

In [28]:
prompt_template="""Use the following pieces of information to answer the user's question.
If you don't know the correct answer, just say you don't know. Don't try to make up an answer.

Context: {context}
Question: {question}

Only return helpful answer below and nothing else
Helful answer:
"""

In [29]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [30]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature': 0.8})

In [31]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs
)

In [32]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response :", result["result"])

/home/marklar/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response : A bank is a grouping of patterns, where each pattern contains sequencer data like trigs, parameter locks, track lengths, and time signatures for the eight audio tracks and the eight MIDI tracks.
Response : Banks and patterns are related in that each bank contains a set of 16 patterns, which are stored within the bank. When switching between banks, the current pattern is retained, meaning that the sequence of events and the parameters set for that pattern will continue to play back uninterrupted. This allows you to easily experiment with different sounds and settings without halting or cutting off playback.
Response : Octatrack MKII is a music performance instrument that gives artists the power to perform live in the truest sense of the word. It's a more robust and improved version of its predecessor, with enhanced features such as durable back-lit buttons, upgraded silky smooth crossfader, ultra crisp OLED screen, precise hi-res encoders, balanced audio inputs, and an improv